## ライブラリのインポート

In [1]:
!pip install -q sklearn
from __future__ import absolute_import, division, print_function, unicode_literals

import numpy as np
import pandas as pd

!pip install -q tensorflow==2.0.0-alpha0
import tensorflow as tf

from tensorflow import feature_column
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split

     |████████████████████████████████| 79.9MB 1.2MB/s 
     |████████████████████████████████| 3.0MB 43.7MB/s 
     |████████████████████████████████| 419kB 47.1MB/s 


## 前処理

### データフレーム作成

In [2]:
URL = 'https://storage.googleapis.com/applied-dl/heart.csv'
dataframe = pd.read_csv(URL)
dataframe.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,1,145,233,1,2,150,0,2.3,3,0,fixed,0
1,67,1,4,160,286,0,2,108,1,1.5,2,3,normal,1
2,67,1,4,120,229,0,2,129,1,2.6,2,2,reversible,0
3,37,1,3,130,250,0,0,187,0,3.5,3,0,normal,0
4,41,0,2,130,204,0,2,172,0,1.4,1,0,normal,0


### 訓練用、検証用、テスト用に分割

In [3]:
train, test = train_test_split(dataframe, test_size=0.2)
train, val = train_test_split(train, test_size=0.2)
print(len(train), 'train examples')
print(len(val), 'validation examples')
print(len(test), 'test examples')

193 train examples
49 validation examples
61 test examples


### tf.dataを使った入力パイプラインの構築

In [0]:
# Pandasデータフレームからtf.dataデータセットを作るためのユーティリティメソッド
def df_to_dataset(dataframe, shuffle=True, batch_size=32):
  dataframe = dataframe.copy()
  labels = dataframe.pop('target')
  ds = tf.data.Dataset.from_tensor_slices((dict(dataframe), labels))
  if shuffle:
    ds = ds.shuffle(buffer_size=len(dataframe))
  ds = ds.batch(batch_size)
  return ds

### 入力パイプラインを理解する

In [0]:
batch_size = 5 # デモ用として小さなバッチサイズを使用
train_ds = df_to_dataset(train, batch_size=batch_size)
val_ds = df_to_dataset(val, shuffle=False, batch_size=batch_size)
test_ds = df_to_dataset(test, shuffle=False, batch_size=batch_size)

In [10]:
# データセットが列名からなるdictionaryを返す
for feature_batch, label_batch in train_ds.take(1):
  print('Every feature:', list(feature_batch.keys()))
  print('A batch of ages:', feature_batch['age'])
  print('A batch of targets:', label_batch )

Every feature: ['age', 'sex', 'cp', 'trestbps', 'chol', 'fbs', 'restecg', 'thalach', 'exang', 'oldpeak', 'slope', 'ca', 'thal']
A batch of ages: tf.Tensor([44 45 60 74 43], shape=(5,), dtype=int32)
A batch of targets: tf.Tensor([0 0 1 0 0], shape=(5,), dtype=int32)


### feature columnsの様々な型の例

In [0]:
# いくつかの型のfeature columnsを例示するためこのバッチを使用する
example_batch = next(iter(train_ds))[0]

In [0]:
# feature columnsを作りデータのバッチを変換する
# ユーティリティメソッド
def demo(feature_column):
  feature_layer = layers.DenseFeatures(feature_column)
  print(feature_layer(example_batch).numpy())

#### 数値コラム

In [13]:
age = feature_column.numeric_column("age")
demo(age)

Instructions for updating:
Use `tf.cast` instead.
[[44.]
 [45.]
 [60.]
 [74.]
 [43.]]


#### パケット化コラム
数値をそのままモデルに入力するのではなく、値の範囲に基づいた異なるカテゴリーに分割したいことがあります。
例えば、人の年齢を表す生データを考えてみましょう。
パケット化コラムを使うと年齢を数値コラムとして表現するのではなく、年齢をいくつかのパケットに分割できます。
下記の`one-hot`値が、各業がどの年齢範囲にあるかを表していることに注目してください。

In [14]:
age_buckets = feature_column.bucketized_column(age, boundaries=[18, 25, 30, 35, 40, 45, 50, 55, 60, 65])
demo(age_buckets)

Instructions for updating:
Use `tf.cast` instead.
[[0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]]


#### カテゴリー型コラム
このデータセットでは、Thalは（'fixed'、'normal'、'reversible'のような）文字列として表現されています。
文字列を直接モデルに入力することはできません。まず、文字列を数値にマッピングする必要があります。
categorical vocabulary コラムを使うと、（上記で示した年齢バケットのように）文字列をワンホットベクトルとして表現することができます。
カテゴリーを表す語彙(vocabulary)はcategorical_column_with_vocabulary_listを使ってリストで渡すか、
categorical_column_with_vocabulary_fileを使ってファイルから読み込むことができます。

In [15]:
thal = feature_column.categorical_column_with_vocabulary_list(
      'thal', ['fixed', 'normal', 'reversible'])

thal_one_hot = feature_column.indicator_column(thal)
demo(thal_one_hot)

Instructions for updating:
The old _FeatureColumn APIs are being deprecated. Please use the new FeatureColumn APIs instead.
Instructions for updating:
The old _FeatureColumn APIs are being deprecated. Please use the new FeatureColumn APIs instead.
[[0. 1. 0.]
 [0. 1. 0.]
 [0. 0. 1.]
 [0. 1. 0.]
 [0. 1. 0.]]


#### 埋込み型コラム
数種類の候補となる文字列ではなく、カテゴリー毎に数千（あるいはそれ以上）の値があるとしましょう。カテゴリーの数が多くなってくると、様々な理由から、ワンホットエンコーディングを使ってニューラルネットワークを訓練することが難しくなります。埋込み型コラムを使うと、こうした制約を克服することが可能です。埋込み型コラムは、データを多次元のワンホットベクトルとして表すのではなく、セルの値が0か1かだけではなく、どんな数値でもとれるような密な低次元ベクトルとして表現します。埋め込みのサイズ（下記の例では8）は、チューニングが必要なパラメータです。

キーポイント：カテゴリー型コラムがたくさんの選択肢を持つ場合、埋め込み型コラムを使用することが最善の方法です。ここでは例を一つ示しますので、今後様々なデータセットを扱う際には、この例を参考にしてください。

In [18]:
# この埋込み型コラムの入力は、先程作成したカテゴリ型コラムであることに注意
thal_embedding = feature_column.embedding_column(thal, dimension=8)
demo(thal_embedding)

[[ 0.0740043   0.58789414 -0.10886434  0.14910814  0.54187363  0.40283918
  -0.10815587 -0.12399125]
 [ 0.0740043   0.58789414 -0.10886434  0.14910814  0.54187363  0.40283918
  -0.10815587 -0.12399125]
 [-0.52580845 -0.18480971 -0.02092033  0.3416489   0.13081387 -0.4303618
  -0.3260405  -0.24808891]
 [ 0.0740043   0.58789414 -0.10886434  0.14910814  0.54187363  0.40283918
  -0.10815587 -0.12399125]
 [ 0.0740043   0.58789414 -0.10886434  0.14910814  0.54187363  0.40283918
  -0.10815587 -0.12399125]]


#### ハッシュ化特徴コラム
値の種類が多いカテゴリー型コラムを表現するもう一つの方法が、categorical_column_with_hash_bucketを使う方法です。このfeature columnは文字列をエンコードするために入力のハッシュ値を計算し、hash_bucket_size個のバケットの中から1つを選択します。このコラムを使用する場合には、語彙を用意する必要はありません。また、スペースの節約のために、実際のカテゴリー数に比べて極めて少ないバケット数を選択することも可能です。

キーポイント：この手法の重要な欠点の一つは、異なる文字列が同じバケットにマッピングされるというハッシュ値の衝突が起きることです。実務上は、データセットによっては、この問題を無視できることがあります。

In [22]:
thal_hashed = feature_column.categorical_column_with_hash_bucket(
      'thal', hash_bucket_size=1000)
demo(feature_column.indicator_column(thal_hashed))

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


#### クロスフィーチャーコラム
複数の特徴量をまとめて1つの特徴量にする、フィーチャークロスとして知られている手法は、モデルが特徴量の組み合わせの一つ一つに別々の重みを学習することを可能にします。ここでは年齢とThalをクロスさせて新しい特徴量を作ってみます。交差列(crossed_column)が、起こりうるすべての組み合わせ全体のテーブル（これは非常に大きくなる可能性があります）を作るものではないことに注意してください。クロスフィーチャーコラムは、代わりにバックエンドとしてハッシュ化コラムを使用しているため、テーブルの大きさを選択することができます。

In [23]:
crossed_feature = feature_column.crossed_column([age_buckets, thal], hash_bucket_size=1000)
demo(feature_column.indicator_column(crossed_feature))

Instructions for updating:
The old _FeatureColumn APIs are being deprecated. Please use the new FeatureColumn APIs instead.
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


### 使用するコラムを選択する

In [0]:
feature_columns = []

# 数値コラム
for header in ['age', 'trestbps', 'chol', 'thalach', 'oldpeak', 'slope', 'ca']:
  feature_columns.append(feature_column.numeric_column(header))

# バケット化コラム
age_buckets = feature_column.bucketized_column(age, boundaries=[18, 25, 30, 35, 40, 45, 50, 55, 60, 65])
feature_columns.append(age_buckets)

# インジケーター（カテゴリー型）コラム
thal = feature_column.categorical_column_with_vocabulary_list(
      'thal', ['fixed', 'normal', 'reversible'])
thal_one_hot = feature_column.indicator_column(thal)
feature_columns.append(thal_one_hot)

# 埋め込み型コラム
thal_embedding = feature_column.embedding_column(thal, dimension=8)
feature_columns.append(thal_embedding)

# クロスフィーチャーコラム
crossed_feature = feature_column.crossed_column([age_buckets, thal], hash_bucket_size=1000)
crossed_feature = feature_column.indicator_column(crossed_feature)
feature_columns.append(crossed_feature)

### 特徴量層の構築

In [0]:
feature_layer = tf.keras.layers.DenseFeatures(feature_columns)

In [0]:
batch_size = 32
train_ds = df_to_dataset(train, batch_size=batch_size)
val_ds = df_to_dataset(val, shuffle=False, batch_size=batch_size)
test_ds = df_to_dataset(test, shuffle=False, batch_size=batch_size)

## モデルの構築

In [27]:
model = tf.keras.Sequential([
  feature_layer,
  layers.Dense(128, activation='relu'),
  layers.Dense(128, activation='relu'),
  layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.fit(train_ds, 
          validation_data=val_ds, 
          epochs=5)

Epoch 1/5
7/7 [==============================] - 1s 100ms/step - loss: 3.3514 - accuracy: 0.4562 - val_loss: 2.5785 - val_accuracy: 0.7347
Epoch 2/5
7/7 [==============================] - 0s 30ms/step - loss: 1.9427 - accuracy: 0.6667 - val_loss: 0.3920 - val_accuracy: 0.7755
Epoch 3/5
7/7 [==============================] - 0s 30ms/step - loss: 1.3353 - accuracy: 0.7483 - val_loss: 1.4210 - val_accuracy: 0.7347
Epoch 4/5
7/7 [==============================] - 0s 30ms/step - loss: 1.0255 - accuracy: 0.6936 - val_loss: 0.5860 - val_accuracy: 0.7347
Epoch 5/5
7/7 [==============================] - 0s 30ms/step - loss: 0.8285 - accuracy: 0.6979 - val_loss: 0.6656 - val_accuracy: 0.7551


In [28]:
loss, accuracy = model.evaluate(test_ds)
print('Accuracy', accuracy)

2/2 [==============================] - 0s 21ms/step - loss: 0.6328 - accuracy: 0.7377
Accuracy 0.73770493
